In [79]:
import numpy as np
import scipy

In [80]:
def max_eigh(D,c):
    pass

def symmetric(X):
    pass

def dDiag(X):
    pass

In [81]:
def vl_myfc(X, W, dzdy=None):
    if dzdy is None:
        return W.T.dot(X_t)
    else:
        Y = W.dot(dzdy)
        Y_w = X.dot(dzdy.T)
        return Y, Y_w

In [82]:
def vl_myrec(X, epsilon, dzdy=None):
    svd_u = np.zeros((X.shape[2], X.shape[0], X.shape[0]))
    svd_s = np.zeros((X.shape[2], np.min(X.shape)))
    svd_v = np.zeros((X.shape[2], X.shape[1], X.shape[1]))
    
    for i in range(X.shape[0]):
        u, s, v = np.linalg.svd(X[i,:,:])
        svd_u[i,:,:] = u
        svd_s[i,:] = s
        svd_v[i,:,:] = v
    
    answer = input.data.numpy()
    if dzdy is None:
        for i, data in enumerate(X):
            eigenvalues, eigenvectors = np.linalg.eig(data)
            eigenvalues = np.maximum(eigenvalues, epsilon)
            answer[i,:,:] = eigenvectors.dot(np.diag(eigenvalues).dot(eigenvectors.T))
        return answer
    else:
        for i in range(X.shape[0]):
            U, S, V = svd_u[i], np.diag(svd_s[i]), svd_v[i]
            
            Dmin = S.shape[0]
            
            dLdC = symmetric(dLdC)
            
            max_S, max_I = my_eigh(S, epsilon)
            dLdV = 2 * dLdC.dot(U).dot(max_S)
            dLdS = np.diag(np.where(dLdV == 0, dLdV))
            dLdS = dLdS.dot(U.T).dot(dLdC).dot(U)
            
            K = diag(S).reshape(-1,)
            K = np.repeat(K.reshape(1,-1), K.shape[0], axis=0) - np.repeat(K.reshape(-1,1), K.shape[0], axis=1)
            K = np.where(np.abs(K) < 1e-6, np.inf, K)
            K = 1. / K
            
            #dzdx = U*(symmetric(K'.*(U'*dLdV))+dDiag(dLdS))*U';
            #Y{ix} =  dzdx; %warning('no normalization');
            
            dzdx = U.dot(symmetric(K.T.dot(U.T.dot(dLdV)) + dDiag(dLdS))).dot(U.T)
            answer[i, :, :] = dzdx

In [90]:
def vl_mysoftmaxloss(X,c,dzdy):
    output = np.subtract(X, X.max(axis=0, keepdims=True))
    logarithms = np.log(np.sum(np.exp(output), axis=0))
    
    if dzdy is None:
        output = output - logarithms.reshape(-1,1)
        return output
    else:
        dzdy = dzdy - np.exp(output) * (np.sum(dzdy, axis=0) / np.sum(np.exp(output), axis=0)).reshape(-1,1)

In [ ]:
def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        
        # Your code goes here. ################################################
        logarithms = np.log(np.sum(np.exp(self.output), axis=1))
        self.output = self.output - logarithms.reshape(-1,1)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        
        self.gradInput = gradOutput - np.exp(self.output) * (np.sum(gradOutput, axis=1) / np.sum(np.exp(self.output), axis=1)).reshape(-1,1)
        return self.gradInput

In [13]:
X = np.arange(27).reshape(3,3,3)
for i, data in enumerate(X):
    print(i, data)
    print('-----')

0 [[0 1 2]
 [3 4 5]
 [6 7 8]]
-----
1 [[ 9 10 11]
 [12 13 14]
 [15 16 17]]
-----
2 [[18 19 20]
 [21 22 23]
 [24 25 26]]
-----


In [24]:
A = np.arange(16).reshape(4,4)
w, e = np.linalg.eig(A)

In [15]:
np.linalg.matrix_rank(A)

2

In [62]:
a = np.array([1,2,3,1,4,2,1000])

In [74]:
aa = np.repeat(a.reshape(1,-1), a.shape[0], axis=0) - np.repeat(a.reshape(-1,1), a.shape[0], axis=1)

In [64]:
args = np.argwhere(aa != 0)

In [65]:
aa[args[:, 0], args[:, 1]] = 1.0 / aa[args[:, 0], args[:, 1]]

In [66]:
aa

array([[ 0,  1,  0,  0,  0,  1,  0],
       [-1,  0,  1, -1,  0,  0,  0],
       [ 0, -1,  0,  0,  1, -1,  0],
       [ 0,  1,  0,  0,  0,  1,  0],
       [ 0,  0, -1,  0,  0,  0,  0],
       [-1,  0,  1, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]])

In [76]:
aa[args[:, 0], args[:, 1]]

array([   1,    2,    3,    1,  999,   -1,    1,   -1,    2,  998,   -2,
         -1,   -2,    1,   -1,  997,    1,    2,    3,    1,  999,   -3,
         -2,   -1,   -3,   -2,  996,   -1,    1,   -1,    2,  998, -999,
       -998, -997, -999, -996, -998])

In [75]:
aa

array([[   0,    1,    2,    0,    3,    1,  999],
       [  -1,    0,    1,   -1,    2,    0,  998],
       [  -2,   -1,    0,   -2,    1,   -1,  997],
       [   0,    1,    2,    0,    3,    1,  999],
       [  -3,   -2,   -1,   -3,    0,   -2,  996],
       [  -1,    0,    1,   -1,    2,    0,  998],
       [-999, -998, -997, -999, -996, -998,    0]])

In [77]:
aa = np.where(np.abs(aa) < 1e-6, np.inf, aa)

In [78]:
1 / aa

array([[ 0.        ,  1.        ,  0.5       ,  0.        ,  0.33333333,
         1.        ,  0.001001  ],
       [-1.        ,  0.        ,  1.        , -1.        ,  0.5       ,
         0.        ,  0.001002  ],
       [-0.5       , -1.        ,  0.        , -0.5       ,  1.        ,
        -1.        ,  0.00100301],
       [ 0.        ,  1.        ,  0.5       ,  0.        ,  0.33333333,
         1.        ,  0.001001  ],
       [-0.33333333, -0.5       , -1.        , -0.33333333,  0.        ,
        -0.5       ,  0.00100402],
       [-1.        ,  0.        ,  1.        , -1.        ,  0.5       ,
         0.        ,  0.001002  ],
       [-0.001001  , -0.001002  , -0.00100301, -0.001001  , -0.00100402,
        -0.001002  ,  0.        ]])

In [85]:
from scipy.io import loadmat

In [87]:
data = loadmat('/Users/alex/Downloads/spdface_400_inter_histeq/train/1/1_1.mat')

In [89]:
data['Y1'].shape

(400, 400)